# Azure AI Agent service - Connected agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ConnectedAgentTool, MessageRole, ListSortOrder, BingGroundingTool
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 16-Jun-2025 14:39:17


## Settings

In [4]:
load_dotenv("azure.env")

True

In [5]:
endpoint = os.getenv("PROJECT_ENDPOINT")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

In [6]:
print(dir(project_client))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_client', '_config', '_delete_agent', '_deserialize', '_function_tool', '_function_tool_max_retry', '_is_protocol', '_serialize', '_validate_tools_and_tool_resources', 'close', 'create_agent', 'create_thread_and_process_run', 'create_thread_and_run', 'delete_agent', 'enable_auto_function_calls', 'files', 'get_agent', 'list_agents', 'messages', 'run_steps', 'runs', 'send_request', 'threads', 'update_agent', 'vector_store_file_batches', 'vector_store_files', 'vector_stores']


## Agents definition

### 1. Stocks price agent

In [7]:
BING_CONN_ID = os.getenv("BING_CONN_ID")

bing = BingGroundingTool(connection_id=BING_CONN_ID)
bing

In [8]:
stocksprice_agent = project_client.create_agent(
    model="gpt-4o",
    name="stocks-price-agent-demo",
    instructions="Retrieve the current stock price for a specified company.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Stocks price agent (demo)",
)

In [9]:
print(f"Created agent, ID: {stocksprice_agent.id}")

Created agent, ID: asst_rcxMbpHnHicUdR2LxzwzDwDB


### 2. Currency agent

In [10]:
currency_convert_agent = project_client.create_agent(
    model="gpt-4o",
    name="currency-convert-agent-demo",
    instructions="Retrieve the current currency exchange rate.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
    description="Currency convert agent (demo)",
)

In [11]:
print(f"Created agent, ID: {currency_convert_agent.id}")

Created agent, ID: asst_MFYjjAZiLiFPm49nopCD9vv8


### 3. Main agent with connected agents

In [12]:
connected_agent_stocks = ConnectedAgentTool(
    id=stocksprice_agent.id,
    name="stockspriceagentconnection",
    description="Get the stock price of a company")

In [13]:
connected_agent_currency = ConnectedAgentTool(
    id=currency_convert_agent.id,
    name="currencyconvertagentconnection",
    description="Convert an amount into another currency")

In [14]:
main_agent_demo = project_client.create_agent(
    model="gpt-4o",
    name="main-agent-demo",
    instructions="You are the main AI agent",
    description="Main agent (demo)",
    tools=[
        connected_agent_stocks.definitions[0],
        connected_agent_currency.definitions[0]
    ],
)

In [15]:
print(f"Created agent, ID: {main_agent_demo.id}")

Created agent, ID: asst_wAvUGrOEa7SYtJO7WNOBHFga


## List of agents

In [19]:
agents_list = [stocksprice_agent, currency_convert_agent, main_agent_demo]

for agent in agents_list:
    print(
        f"Agent Details:\n"
        f"📛 Name        : {agent.name}\n"
        f"🆔 ID          : {agent.id}\n"
        f"🧠 Model       : {agent.model}\n"
        f"📜 Instructions:{agent.instructions}\n"
    )

Agent Details:
📛 Name        : stocks-price-agent-demo
🆔 ID          : asst_rcxMbpHnHicUdR2LxzwzDwDB
🧠 Model       : gpt-4o
📜 Instructions:Retrieve the current stock price for a specified company.

Agent Details:
📛 Name        : currency-convert-agent-demo
🆔 ID          : asst_MFYjjAZiLiFPm49nopCD9vv8
🧠 Model       : gpt-4o
📜 Instructions:Retrieve the current currency exchange rate.

Agent Details:
📛 Name        : main-agent-demo
🆔 ID          : asst_wAvUGrOEa7SYtJO7WNOBHFga
🧠 Model       : gpt-4o
📜 Instructions:You are the main AI agent



### Quick test

In [20]:
prompt = "Hello. Who are you?"

In [21]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=stocksprice_agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_oEjV7cf42l4WHPumdm55Fho1
💬 [2] Created message | ID: msg_R258C5to3QZMFpTCrjkr2KcP
🏃 [3] Created run | ID: run_X7gC4fzKOKnVZwAvJU2tEbxe
⏳ [4] Polling run status...
🔄 Status: completed

🤖 [5] Agent responses:

💡 → Hello. Who are you?

💡 → Hello! I am an AI assistant here to help you with information, answer questions, and assist you with tasks. How can I help you today?



### Question 1

In [22]:
prompt = "What is the current value of the Microsoft stock?"

In [23]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=stocksprice_agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_jcqzmsJBlF7hp1nZ2ZTCyM47
💬 [2] Created message | ID: msg_G5QQyeKlvKa6k9d8YJ8sB0XV
🏃 [3] Created run | ID: run_ltrvC9oXWafO8z2XHux0vtev
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What is the current value of the Microsoft stock?

💡 → The current stock price of Microsoft (MSFT) as of June 12, 2025, is $478.87【3:1†source】【3:4†source】.

🔗 URL Citation: [Microsoft (Nasdaq:MSFT) Historical Data | Historical Stock Price Data ...](https://markets.financialcontent.com/stocks/quote/historical?Symbol=537%3A951692)
🔗 URL Citation: [Microsoft (MSFT) Stock Price History & Chart Since 1986](https://wallstreetnumbers.com/stocks/msft/price)



In [24]:
run.usage

{'prompt_tokens': 7407, 'completion_tokens': 63, 'total_tokens': 7470, 'prompt_token_details': {'cached_tokens': 0}}

### Question 2

In [25]:
prompt = "Convert this into €"

In [26]:
# Step 1: Use the previous thread
print(f"🧵 [1] Using the existing thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=stocksprice_agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Using the existing thread | ID: thread_jcqzmsJBlF7hp1nZ2ZTCyM47
💬 [2] Created message | ID: msg_g2J5OjJVKsSHi9cwfWkAOkpD
🏃 [3] Created run | ID: run_KpgqZfSz07OAMBwevtklVrzs
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What is the current value of the Microsoft stock?

💡 → The current stock price of Microsoft (MSFT) as of June 12, 2025, is $478.87【3:1†source】【3:4†source】.

🔗 URL Citation: [Microsoft (Nasdaq:MSFT) Historical Data | Historical Stock Price Data ...](https://markets.financialcontent.com/stocks/quote/historical?Symbol=537%3A951692)
🔗 URL Citation: [Microsoft (MSFT) Stock Price History & Chart Since 1986](https://wallstreetnumbers.com/stocks/msft/price)

💡 → Convert this into €

💡 → The current exchange rate is 1 USD = €0.8735【7:2†source】. At this rate, the Microsoft stock price of $478.87 converts to approximately €418.14.

🔗 URL Citation: [Exchange Rates History for 5 June 2025 (05/06/2025)](https://www.exchangera

### Question 3

In [27]:
prompt = "I have 100 Microsoft shares, 50 IBM and 75 SFDC shares. What is the total value of my portfolio in eur?"

In [28]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=stocksprice_agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_FCITfxZxtyDnV6Pwc9FHNBuZ
💬 [2] Created message | ID: msg_iSp7K7vWQq4Q1LkfGx0iEeyc
🏃 [3] Created run | ID: run_dMWAcDfjsKxR9Aa3X4nm5Lyl
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → I have 100 Microsoft shares, 50 IBM and 75 SFDC shares. What is the total value of my portfolio in eur?

💡 → The current stock prices for your shares are as follows:
- Microsoft (MSFT): $474.96【3:0†source】.
- IBM: $277.22【5:3†source】.
- Salesforce (SFDC): $266.91【7:1†source】.

Calculating the portfolio value in USD:
- Microsoft: 100 shares × $474.96 = $47,496
- IBM: 50 shares × $277.22 = $13,861
- Salesforce: 75 shares × $266.91 = $20,018.25

Total value in USD: $47,496 + $13,861 + $20,018.25 = $81,375.25.

To convert this to EUR, the current exchange rate would be needed. Could you provide me with the latest USD to EUR rate, or should 

## Post processing

In [32]:
agents_list = [stocksprice_agent, currency_convert_agent, main_agent_demo]

for agent in agents_list:
    print(
        f"Agent Details:\n"
        f"📛 Name        : {agent.name}\n"
        f"🆔 ID          : {agent.id}\n"
        f"🧠 Model       : {agent.model}\n"
        f"📜 Instructions: {agent.instructions}\n"
    )

Agent Details:
📛 Name        : stocks-price-agent-demo
🆔 ID          : asst_rcxMbpHnHicUdR2LxzwzDwDB
🧠 Model       : gpt-4o
📜 Instructions: Retrieve the current stock price for a specified company.

Agent Details:
📛 Name        : currency-convert-agent-demo
🆔 ID          : asst_MFYjjAZiLiFPm49nopCD9vv8
🧠 Model       : gpt-4o
📜 Instructions: Retrieve the current currency exchange rate.

Agent Details:
📛 Name        : main-agent-demo
🆔 ID          : asst_wAvUGrOEa7SYtJO7WNOBHFga
🧠 Model       : gpt-4o
📜 Instructions: You are the main AI agent



In [ ]:
#print("Deleting the agents we have created...")
#for agent in agents_list:
    #project_client.delete_agent(agent.id)
    #print(f"Deleted agent, agent ID: {agent.id}")
#print("Done")